In [19]:
from myml.nbinit import *

from etl import grapher_model as gm
from etl.config import OWIDEnv, OWID_ENV
from sqlalchemy.orm import Session
from etl.db import read_sql

# engine = OWIDEnv.from_staging("master").get_engine()

engine = OWID_ENV.get_engine()

# read_sql("""
# select
#   v.name,
#   COALESCE(v.description, v.descriptionShort) as description,
#   count(distinct cd.chartId) as n_charts
# from variables as v
# join datasets as d on v.datasetId = d.id
# left join chart_dimensions as cd on v.id = cd.variableId
# where d.isArchived = 0
# group by 1, 2
# limit 10;
# """)

df = read_sql("""
with t as (
    select
        v.id as variableId,
        v.name,
        COALESCE(v.description, v.descriptionShort) as description
    from variables as v
    join datasets as d on v.datasetId = d.id
    where d.isArchived = 0
), n_charts as (
    select
        variableId,
        count(distinct chartId) as n_charts
    from chart_dimensions as cd
    group by 1
)
select
    t.variableId,
    t.name,
    t.description,
    COALESCE(n_charts.n_charts, 0) as n_charts
from t
left join n_charts on t.name = n_charts.variableId
""")

In [20]:
df

,variableId,name,description,n_charts
0,27,Human Height (University of Tuebingen (2015)),Human height measured in centimetres.,0
1,35,Historical European trade (Brian Mitchell (2015)),,0
2,39,IIASA rates of no education projections (IIASA...,No education includes Illiterates and persons ...,0
3,73,Total number of people aged 15+ with no educat...,No education includes Illiterates and persons ...,0
4,110,"No education, 1970-2050 (IIASA (2015))",No education includes: Illiterates and persons...,0
...,...,...,...,...
183767,997286,Deaths from famines where external factors was...,"Famines that are estimated to have killed 100,...",0
183768,997287,Deaths from famines,"Famines that are estimated to have killed 100,...",0
183769,997288,Region,"Famines that are estimated to have killed 100,...",0
183770,997289,GDP per capita,"Famines that are estimated to have killed 100,...",0


In [6]:
size = 100000

url = f'http://analytics/analytics.csv?sql=select+indicator_id%2C+name%2C+n_charts%2C+views_365d+from+indicators+limit+{size}&_size=max'
df = pd.read_csv(url)

In [11]:
df

,indicator_id,name,n_charts,views_365d
0,132,Number of peacekeeping missions (United Nation...,1,1205
1,306,"Fertility rate (Complete Gapminder, v12) (2017)",1,6375
2,1455,Gini equivalised disposable household income e...,0,0
3,2066,CO2 emissions from manufacturing industries an...,0,0
4,2090,Fertilizer consumption (kilograms per hectare ...,0,0
...,...,...,...,...
3995,467735,Indicator:Mental hospital admissions (per 100 ...,0,0
3996,467755,Indicator:Adolescent mortality rate (per 100 0...,0,0
3997,467807,Indicator:Prevalence of overweight among child...,0,0
3998,467836,Indicator:General availability of total choles...,0,0
